# Реализация всех методов

## Итерационные методы


## Метод Якоби

In [3]:
import numpy as np


# Бесконечная норма матрицы
def InfinityNormMatrix(A):
    max = 0
    n = A.shape[0]
    for i in range(n):
        sum = 0
        for j in range(n):
            sum += abs(A[i][j])
        if sum > max:
            max = sum
    return max

def InfinityNormVector(x):
    n = x.shape[0]
    max = abs(x[0])
    for i in range(n):
        if abs(x[i]) > max:
            max = abs(x[i])
    return max

# Получаем матрицу B и вектор c, необходимые в методе Якобы
def GetIterationParams(A, b):
    n = A.shape[0]
    B = np.eye(n)
    c = np.zeros(n);
    for i in range(n):
        for j in range(n):
            if i == j:
                B[i][j] = 0
            else:
                B[i][j] = -A[i][j] / A[i][i]
        c[i] = b[i] / A[i][i]
    return B, c
    
    
def MethodJacobi(A, x_0, eps, b):
    B, c = GetIterationParams(A, b)
    x_n_1 = x_0
    x_n = B @ x_n_1 + c
    q = InfinityNormMatrix(B)
    eps = ((1 - q) / q) * eps
    while InfinityNormVector(x_n - x_n_1) > eps:
        x_n_1 = x_n
        x_n = B @ x_n_1 + c
    return x_n

A = np.array([[46, 2, 4, 1], [7, 94, 5, 1], [5, 5, 100, 1], [4, 5, 6, 92]])
x_0 = np.array([1, 1, 1, 1])
b = np.array([198, 914, -735, -19])
x = MethodJacobi(A, x_0, 10**(-5), b)
print("MethodJacobi = ", x)

MethodJacobi =  [ 4.58797893  9.81518758 -8.06602517 -0.41338958]


## Метод Зейделя

In [4]:
# Получение матриц B1 и B2
def GetMatrixB1AndB2(B):
    n = B.shape[0]
    B1 = np.eye(n)
    B2 = np.eye(n)
    for i in range(n):
        for j in range(n):
            if i == j:
                B1[i][j] = 0
                B2[i][j] = 0
            elif i > j:
                B2[i][j] = B[i][j]
            else:
                B1[i][j] = B[i][j]
    return B1, B2



def CalculateXn(B, x_old, c):
    n = B.shape[0]
    x = np.zeros(n)
    m = 0
    for i in range(n):
        x[i] = 0
        for j in range(m):
            x[i] += B[i][j] * x[j]
        for j in range(m, n):
            x[i] += B[i][j] * x_old[j]
        m += 1
        x[i] += c[i]
    return x
        

def MethodSeidel(A, x_0, eps, b):
    B, c = GetIterationParams(A, b)
    B1, B2 = GetMatrixB1AndB2(B)
    eps = eps * (1 - InfinityNormMatrix(B)) / (InfinityNormMatrix(B2))
    x_n_1 = x_0
    x_n = CalculateXn(B, x_n_1, c)
    while InfinityNormVector(x_n - x_n_1) > eps:
        x_n_1 = x_n
        x_n = CalculateXn(B, x_n_1, c)
    return x_n

# Тестовый пример такой же как для метода Якоби
A = np.array([[46, 2, 4, 1], [7, 94, 5, 1], [5, 5, 100, 1], [4, 5, 6, 92]])
x_0 = np.array([1, 1, 1, 1])
b = np.array([198, 914, -735, -19])
x = MethodSeidel(A, x_0, 10**(-5), b)
print("MethodSeidel = ", x)

MethodSeidel =  [ 4.58798063  9.81518772 -8.06602454 -0.41338819]


## Метод релаксации

In [11]:
# Вычисление следующего приближения
def CalculateXn(B, x_old, omega, c):
    n = B.shape[0]
    x = np.zeros(n)
    m = 0
    for i in range(n):
        x[i] = 0
        for j in range(m):
            x[i] += omega * B[i][j] * x[j]
        for j in range(m, n):
            x[i] += omega * B[i][j] * x_old[j]
        m += 1
        x[i] += omega * c[i]
        x[i] += (1 - omega) * x_old[i]
    return x

def SOR(A, x_0, omega, eps, b):
    B, c = GetIterationParams(A, b)
    B1, B2 = GetMatrixB1AndB2(B)
    eps = eps * (1 - InfinityNormMatrix(B)) / (InfinityNormMatrix(B2))
    x_n_1 = x_0
    x_n = CalculateXn(B, x_n_1, omega, c)
    while InfinityNormVector(x_n - x_n_1) > eps:
        x_n_1 = x_n
        x_n = CalculateXn(B, x_n_1, omega, c)
    return x_n

# Тестовый пример такой же как для метода Якоби
A = np.array([[46, 2, 4, 1], [7, 94, 5, 1], [5, 5, 100, 1], [4, 5, 6, 92]])
x_0 = np.array([1, 1, 1, 1])
b = np.array([198, 914, -735, -19])
omega = 1.7882
x = SOR(A, x_0, omega, 10**(-5), b)
print("SOR = ", x)

SOR =  [ 4.58798233  9.81519215 -8.06603273 -0.41336644]


## Прямые методы

## Метод Холецкого 

In [6]:
def Transpose(A):
    n = A.shape[0]
    A_t = np.eye(n)
    for i in range(n):
        for j in range(n):
            A_t[i][j] = A[j][i]
    return A_t

def DecompositionCholecy(A):
    n = A.shape[0]
    L = np.eye(n)
    for i in range(n):
        s = A[i][i]
        for k in range(i):
            s -= L[i][k]**2
        L[i][i] = np.sqrt(s)
        for j in range(i + 1, n):
            s = A[j][i]
            for k in range(i):
                s -= L[i][k] * L[j][k]
            L[j][i] = s / L[i][i]
    L_t = Transpose(L.copy())
    return L, L_t

# Проверка корректности разложения
A = np.array([[25, 30, 10], [30, 40, 30], [10, 30, 110]])
L, L_t = DecompositionCholecy(A)
print("L = ", L)
print("L^T = ", L_t)

L =  [[5. 0. 0.]
 [6. 2. 0.]
 [2. 9. 5.]]
L^T =  [[5. 6. 2.]
 [0. 2. 9.]
 [0. 0. 5.]]


In [7]:
# Метод решающий систему на основе данного разложения
def SolveSystemUsingCholecy(L, L_t, b):
    n = L.shape[0]
    y = np.zeros(n)
    y[0] = b[0] / L[0][0]
    for i in range(1, n):
        y[i] = b[i]
        for j in range(0, i):
            y[i] -= L[i][j] * y[j]
        y[i] /= L[i][i]
    x = np.zeros(n)
    x[n - 1] = y[n - 1] / L_t[n - 1][n - 1]    
    for i in range(n - 2, -1, -1):
        cur = y[i]
        for j in range(n - 1, i, -1):
            cur -= L_t[i][j] * x[j]
        x[i] = cur / L_t[i][i]
    return x

b = np.array([-95, -270, -990])
x =  SolveSystemUsingCholecy(L, L_t, b)
print("Method Cholecy = ", x)

Method Cholecy =  [ -7.   6. -10.]


## Метод Гаусса (классическая схема)

In [13]:
# Прямой ход метода Гаууса
def ForwardStrake(A, b):
    n = A.shape[0]
    for k in range(n - 1):
        for i in range(k + 1, n):
            mu_i_k = A[i][k] / A[k][k]
            for j in range(k, n):
                A[i][j] -= (mu_i_k * A[k][j])
            b[i] -= (b[k] * mu_i_k)   
    return A, b

# Обратный ход метода Гаууса
def BackStrake(A, b):
    n = A.shape[0]
    x = np.zeros(n)
    x[n - 1] = b[n - 1] / A[n - 1][n - 1]
    for i in range(n - 2, -1, -1):
        s = b[i]
        for j in range(n - 1, i, -1):
            s -= (A[i][j] * x[j])
        x[i] = s / A[i][i]
    return x

# Метод Гаусса реализация классического варианта
def MethodGauss(A, b):
    A, b = ForwardStrake(A, b)
    x = BackStrake(A, b)
    return x

# Метод Гаусса тест
A = np.array([[9, 0, -11], [-9, 8, 6], [18, -80, 32]])
b = np.array([-13, 11, 2])
x = MethodGauss(A, b)
print("MethodGauss = ", x)

MethodGauss =  [1. 1. 2.]


# Метод Гаусса (схема частичного выбора)

In [17]:
# Находим максимум по модулю в столбце
def MaxElementInRow(A, k):  
    max_item = abs(A[k][k])
    n = A.shape[0]
    for i in range(k + 1, n):
        if abs(A[i][k]) > max_item:
            max_item = abs(A[i][k])
         
    # При выходе из функции возвращаем номер строки, в которой был найден максимальный элемент
    # и сам этот элемент
    for i in range(k, n):
        if abs(A[i][k]) == max_item:
            return i

# Прямой ход метода Гаууса
def ForwardStrake(A, b):
    n = A.shape[0]
    for k in range(n - 1):
        max_row = MaxElementInRow(A, k)
        # Меняем строки местами
        tmp = A[max_row].copy()
        A[max_row] = A[k]
        A[k] = tmp
        tmp = b[max_row].copy()
        b[max_row] = b[k]
        b[k] = tmp
        for i in range(k + 1, n):
            mu_i_k = A[i][k] / A[k][k]
            for j in range(k, n):
                A[i][j] -= (mu_i_k * A[k][j])
            b[i] -= (b[k] * mu_i_k)   
    return A, b

# Обратный ход метода Гаууса
def BackStrake(A, b):
    n = A.shape[0]
    x = np.zeros(n)
    x[n - 1] = b[n - 1] / A[n - 1][n - 1]
    for i in range(n - 2, -1, -1):
        s = b[i]
        for j in range(n - 1, i, -1):
            s -= (A[i][j] * x[j])
        x[i] = s / A[i][i]
    return x

# Метод Гаусса реализация схемы частичного выбора
def MethodGauss(A, b):
    A, b = ForwardStrake(A, b)
    x = BackStrake(A, b)
    return x

# Метод Гаусса тест
A = np.array([[9.0, 0.0, -11.0], [-9.0, 8.0, 6.0], [18.0, -80.0, 32.0]])
b = np.array([-13.0, 11.0, 2.0])
x = MethodGauss(A, b)
print("MethodGauss = ", x)

MethodGauss =  [1. 1. 2.]


## Метод Гаусса (схема полного выбора)

In [67]:
# Находим максимум по модулю в матрицу 
def MaxElementInMatrix(A, k):
    max = 0
    n = A.shape[0]
    for i in range(k, n):
        for j in range(k, n):
            if abs(A[i][j]) > max:
                max = abs(A[i][j])
    for i in range(k, n):
        for j in range(k, n):
            if abs(A[i][j]) == max:
                return i, j
            
# Прямой ход метода Гаууса
def ForwardStrake(A, b):
    n = A.shape[0]
    permutations = []
    for k in range(n - 1):
        max_row, max_column = MaxElementInMatrix(A, k)
        # Меняем строки местами
        tmp = A[max_row].copy()
        A[max_row] = A[k]
        A[k] = tmp
        tmp = b[max_row].copy()
        b[max_row] = b[k]
        b[k] = tmp
        # меняем столбцы местами
        
        permutations.append((k, max_column))
        tmp = np.copy(A[:, k])
        A[:, k] = A[:, max_column]
        A[:, max_column] = tmp
        
        
        for i in range(k + 1, n):
            mu_i_k = A[i][k] / A[k][k]
            for j in range(k, n):
                A[i][j] -= (mu_i_k * A[k][j])
            b[i] -= (b[k] * mu_i_k)   
    return A, b, permutations

# Обратный ход метода Гаууса
def BackStrake(A, b, permutations, n):
    x = np.zeros(n)
    x[n - 1] = b[n - 1] / A[n - 1][n - 1]
    for i in range(n - 2, -1, -1):
        s = b[i]
        for j in range(n - 1, i, -1):
            s -= (A[i][j] * x[j])
        x[i] = s / A[i][i]
    # Теперь строим соответствия для каждого x[i]
    pos = []
    for i in range(n):
        pos.append(i)
    for p in permutations:
        pos_1 = 0
        pos_2 = 0
        for i in range(n):
            if pos[i] == p[0]:
                pos_1 = i
            if pos[i] == p[1]:
                pos_2 = i
        tmp = pos[pos_1]
        pos[pos_1] = pos[pos_2]
        pos[pos_2] = tmp
    ans = x.copy()
    for i in range(n):
        ans[i] = x[pos[i]]
    return ans

# Метод Гаусса реализация схемы полного выбора
def MethodGauss(A, b):
    n = A.shape[0]
    A, b, permutations = ForwardStrake(A, b)
    x = BackStrake(A, b, permutations, n)
    return x

# Метод Гаусса тест
A = np.array([[-4.0, -7.0, -3.0, -2.0], [40.0, 61.0, 29.0, 11.0], [40.0, 7.0, 29.0, -52.0], [24.0, 87.0, 41.0, 25.0]])
b = np.array([-18.0, 238.0, 652.0, 36.0])
x = MethodGauss(A, b)
print("MethodGauss = ", x)

MethodGauss =  [ 8. -3.  5. -4.]


## Метод прогонки

In [34]:
# Метод прогонки, a - массив элементов под главной диагональю, b - массив элементов на главной диагонали
# с - массив элементов над главной диагональю, d - вектор правых частей
def SweepMethod(a, b, c, d):
    n = b.shape[0]
    alpha = np.zeros(n)
    beta = np.zeros(n)
    alpha[0] = -c[0] / b[0]
    beta[0] = d[0] / b[0]
    # Прямой ход метода (было бы лучше вынести в отдельную функцию, но мне лень)
    for i in range(1, n - 1):
        gamma_i = b[i] + a[i] * alpha[i - 1]
        alpha[i] = -c[i] / gamma_i
        beta[i] = (d[i] - a[i] * beta[i - 1]) / gamma_i
    # Обратный ход, тоже бы вынести в отдельную функцию :)
    gamma_n_1 = b[n - 1] + a[n - 1] * alpha[n - 2]
    beta[n - 1] = (d[n - 1] - a[n - 1] * beta[n - 2]) / gamma_n_1
    x = np.zeros(n)
    x[n - 1] = beta[n - 1]
    for i in range(n - 2, -1, -1):
        x[i] = alpha[i] * x[i + 1] + beta[i]
    return x

# Проверка работоспособности метод
# В матрице a обязательно задавать первый элемент любым числом, а дальше уже любая диагональ
a = np.array([0, -6, -3, 1, 5])
b = np.array([4, 20, 7, 6, 8])
c = np.array([2, -4, -1, -3])
d = np.array([-28, -36, 58, 11, 76])
x = SweepMethod(a, b, c, d)
print("SweepMethod = ", x)

SweepMethod =  [-6. -2.  8.  4.  7.]


## LU разложение (классический вариант)

In [35]:
# LU разложение матрицы
def LUDecomposition(A):
    n = A.shape[0]
    U = A # Чтобы не модифицировать матрицу A, копируем ее в U
    L = np.eye(n) # Делаем единичную матрицу n * n
    for k in range(n - 1):
            for i in range(k + 1, n):
                mu_i_k = U[i][k] / U[k][k]
                L[i][k] = mu_i_k
                U[i][k] -= U[k][k] * mu_i_k
                for j in range(k + 1, n):
                    U[i][j] -= (mu_i_k * U[k][j])
    return L, U

# Рещение системы с помощью данного Lu разложения
def SolveSystemUsingLU(L, U, b):
    n = L.shape[0]
    y = np.zeros(n)
    y[0] = b[0]
    for i in range(1, n):
        y[i] = b[i]
        for j in range(0, i):
            y[i] -= L[i][j] * y[j]
            
    x = np.zeros(n)
    x[n - 1] = y[n - 1] / U[n - 1][n - 1]    
    for i in range(n - 2, -1, -1):
        cur = y[i]
        for j in range(n - 1, i, -1):
            cur -= U[i][j] * x[j]
        x[i] = cur / U[i][i]
    return x

A = np.array([[9, 0, -11], [-9, 8, 6], [18, -80, 32]])
b = np.array([-13, 11, 2])
L, U = LUDecomposition(A)
x = SolveSystemUsingLU(L, U, b)
print("LUDecompositionClassic = ", x)

LUDecompositionClassic =  [1. 1. 2.]


## LU разложение (по схеме частичного выбора)

In [38]:
# Находим максимум по модулю в столбце
def MaxElementInRow(A, k):  
    max_item = abs(A[k][k])
    n = A.shape[0]
    for i in range(k + 1, n):
        if abs(A[i][k]) > max_item:
            max_item = abs(A[i][k])
         
    # При выходе из функции возвращаем номер строки, в которой был найден максимальный элемент
    # и сам этот элемент
    for i in range(k, n):
        if abs(A[i][k]) == max_item:
            return i

# LU разложение по схеме частичного выбора
def LUDecomposition(A, b):
    n = A.shape[0]
    L = np.eye(n) # Делаем единичную матрицу n * n
    U = A.copy()
    for k in range(n - 1):
        max_row = MaxElementInRow(U, k)
        # Меняем строки местами
        tmp = U[max_row].copy()
        U[max_row] = U[k]
        U[k] = tmp
        
        tmp = b[max_row].copy()
        b[max_row] = b[k]
        b[k] = tmp
        
        if k > 0: 
            for j in range(k, -1, -1):
                tmp = L[max_row][j].copy()
                L[max_row][j] = L[k][j]
                L[k][j] = tmp
        for i in range(k + 1, n):
            mu_i_k = U[i][k] / U[k][k]
            L[i][k] = mu_i_k
            U[i][k] -= U[k][k] * mu_i_k
            for j in range(k + 1, n):
                U[i][j] -= (mu_i_k * U[k][j])
    return L, U

# Рещение системы с помощью данного LU разложения
def SolveSystemUsingLU(L, U, b):
    n = L.shape[0]
    y = np.zeros(n)
    #print("n = ", n)
    y[0] = b[0]
    for i in range(1, n):
        y[i] = b[i]
        for j in range(0, i):
            y[i] -= L[i][j] * y[j]
            
    x = np.zeros(n)
    x[n - 1] = y[n - 1] / U[n - 1][n - 1]    
    for i in range(n - 2, -1, -1):
        cur = y[i]
        for j in range(n - 1, i, -1):
            cur -= U[i][j] * x[j]
        x[i] = cur / U[i][i]
    return x


A = np.array([[9.0, 0.0, -11.0], [-9.0, 8.0, 6.0], [18.0, -80.0, 32.0]])
b = np.array([-13.0, 11.0, 2.0])
L, U = LUDecomposition(A, b)
x = SolveSystemUsingLU(L, U, b)
print("LUDecompositionSchemeNotFull = ", x)

LUDecompositionSchemeNotFull =  [1. 1. 2.]


## LU разложение (по схеме полного выбора)

## LU разложение (c использованием матрицы перестановок)

In [15]:
# Находим максимум по модулю в столбце
def MaxElementInRow(A, k):  
    max_item = abs(A[k][k])
    n = A.shape[0]
    for i in range(k + 1, n):
        if abs(A[i][k]) > max_item:
            max_item = abs(A[i][k])
         
    # При выходе из функции возвращаем номер строки, в которой был найден максимальный элемент
    # и сам этот элемент
    for i in range(k, n):
        if abs(A[i][k]) == max_item:
            return i

def LUPDecomposition(A):
    n = A.shape[0]
    L = np.eye(n)
    P = np.eye(n)   
    for k in range(n - 1):
        P_k = np.eye(n)
        U_k = np.eye(n)
        L_k = np.eye(n)
        max_row = MaxElementInRow(A, k)
        P_k[max_row], P_k[k] = np.copy(P_k[k]), np.copy(P_k[max_row])
        A = P_k @ A
        for i in range(k + 1, n):
            mu_i_k = A[i][k] / A[k][k]
            U_k[i][k] = -mu_i_k
            L_k[i][k] = mu_i_k
        A = U_k @ A 
        P = P_k @ P
        L = L @ P_k @ L_k
    
    L = P @ L  
    return L, A, P

# Решение системы использование матриц L, U, P
def SolveSystemUsingLUP(L, U, P, b):
    n = L.shape[0]
    y = np.zeros(n)
    b = P @ b
    y[0] = b[0]
    for i in range(1, n):
        y[i] = b[i]
        for j in range(0, i):
            y[i] -= L[i][j] * y[j]
            
    x = np.zeros(n)
    x[n - 1] = y[n - 1] / U[n - 1][n - 1]    
    for i in range(n - 2, -1, -1):
        cur = y[i]
        for j in range(n - 1, i, -1):
            cur -= U[i][j] * x[j]
        x[i] = cur / U[i][i]
    return x

A = np.array([[9.0, 0.0, -11.0], [-9.0, 8.0, 6.0], [18.0, -80.0, 32.0]])
b = np.array([-13.0, 11.0, 2.0])
L, U, P = LUPDecomposition(A)
x = SolveSystemUsingLUP(L, U, P, b)
print("LUPDecomposition = ", x)

LUPDecomposition =  [1. 1. 2.]
